In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\a1831\Desktop\NBS_Project


In [52]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 50
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 0.1, 'l': 0.8, 'cn': 0.540195464528839, 'Fn': 1.9123324177973449, 'kn': 0.2603644452820671, 'omega_n': 1, 'Rn': 7.514263728549031}
{'s': 0.1, 'l': 0.8, 'cn': 0.9869510648376378, 'Fn': 2.6327888013327883, 'kn': 0.7664561149429193, 'omega_n': 1, 'Rn': 9.579412974152985}
{'s': 0.1, 'l': 0.8, 'cn': 0.4084216720608006, 'Fn': 2.4070592699862496, 'kn': 0.15255751936623188, 'omega_n': 1, 'Rn': 9.581415772875358}
{'s': 0.1, 'l': 0.8, 'cn': 0.5983618223126913, 'Fn': 2.2543629859868184, 'kn': 0.8804360367734104, 'omega_n': 1, 'Rn': 8.899050644522536}
{'s': 0.1, 'l': 0.8, 'cn': 0.5513938702825492, 'Fn': 3.096414010656456, 'kn': 0.9194409092602214, 'omega_n': 1, 'Rn': 7.613632009116485}
{'s': 0.1, 'l': 0.8, 'cn': 0.14651023508479327, 'Fn': 2.3082292162520583, 'kn': 0.06527360638806357, 'omega_n': 1, 'Rn': 4.461658670934971}
{'s': 0.1, 'l': 0.8, 'cn': 0.9348335484187288, 'Fn': 3.856370829736375, 'kn': 0.7280338567283863, 'omega_n': 1, 'Rn': 6.396976487364239}
{'s': 0.1, 'l': 0.8, 'cn': 0.46171

In [53]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -318.6253630122395
            Iterations: 6
            Function evaluations: 44
            Gradient evaluations: 2
SWM λ: [4.97301816 4.99670967 4.99109246 4.98651688 5.00522755 4.98821114
 5.01387379 5.01278046 5.01830876 5.01426113]
SWM p: [1.91233242 2.6327888  2.40705927 2.25436299 3.09641401 1.69375934
 3.85637083 3.73282308 4.51222521 3.90300111]
SWM Dmax: 0.9904401514344436
Optimization terminated successfully    (Exit mode 0)
            Current function value: -32.697506999052706
            Iterations: 12
            Function evaluations: 132
            Gradient evaluations: 12
SWM r: [16.74849147 22.5302848  17.09887462 18.67564217 21.70268311 14.09682116
 30.16308921 22.36716973 31.28197495 26.90984582]


In [54]:
print("social welfare:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("sum lambda:", np.sum(lam_swm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_swm[i],lam_swm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_swm, " D0:", esp.D0)
max_Dn = max(md.delay(p_swm[i], lam_swm[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_swm == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_swm[i], lam_swm[i]) <= Dmax_swm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_swm <= esp.D0)
print("pn<=Fn:", all(p_swm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_swm[i] <= p_swm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_swm), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_swm[i] >= md.Ln(p_swm[i]) for i, md in enumerate(MDs)))


social welfare: 163.29903469524575
sum lambda: 50.000000000000014  lambda0: 50
['MD 1 Dn: 0.06613105260920178', 'MD 2 Dn: 0.04626446103000549', 'MD 3 Dn: 0.050282035181861105', 'MD 4 Dn: 0.054353573260285074', 'MD 5 Dn: 0.04280797252840659', 'MD 6 Dn: 0.08420344834415701', 'MD 7 Dn: 0.038785484603551156', 'MD 8 Dn: 0.0350551856100143', 'MD 9 Dn: 0.03540949785306277', 'MD 10 Dn: 0.03252377505923121']
Dmax: 0.9904401514344436  D0: 5
max Dn: 0.08420344834415701
Dmax == max Dn: False
Dn<=Dmax: True
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 221.57487704805874 Q_star: 248.98287084352265
rn>=Ln: True


In [55]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -21.792878338775697
            Iterations: 49
            Function evaluations: 1569
            Gradient evaluations: 49
Optimal λ: [5.01781621 4.96572481 5.03209016 4.97622081 4.97702855 5.04999332
 4.97580452 5.00820804 4.98430422 5.01280936]
Optimal p: [0.45799    0.45370818 0.45901732 0.45457967 0.45472005 0.46093069
 0.45472218 0.45713095 0.45541484 0.45744468]
Optimal r: [20.64810569 20.9407837  20.57601373 20.94633369 20.95231794 20.50094532
 20.88036982 20.74085126 20.86654182 20.63985185]
Optimal Dmax: 1.4276174616152084


In [56]:
print("social welfare:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("sum lambda:", np.sum(lam_opt), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_opt[i],lam_opt[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_opt, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_opt == max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_opt[i], lam_opt[i]) <= Dmax_opt for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_opt <= esp.D0)
print("pn<=Fn:", all(p_opt[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_opt[i] <= p_opt[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_opt), "Q_star:", esp.Q(Dmax_opt))
print("rn>=Ln:", all(r_opt[i] >= md.Ln(p_opt[i]) for i, md in enumerate(MDs)))

social welfare: 245.44918037567504
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 1.4276174618236994', 'MD 2 Dn: 1.427617461887359', 'MD 3 Dn: 1.427617461852763', 'MD 4 Dn: 1.4276174618712285', 'MD 5 Dn: 1.427617461881067', 'MD 6 Dn: 1.4276174619015585', 'MD 7 Dn: 1.4276174619172062', 'MD 8 Dn: 1.4276174618879542', 'MD 9 Dn: 1.4276174618794648', 'MD 10 Dn: 1.4276174618802895']
Dmax: 1.4276174616152084  D0: 5
max Dn: 1.4276174619172062
Dmax == max Dn: False
Dn<=Dmax: False
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 207.69211484050174 Q_star: 248.6003738551861
rn>=Ln: True


In [22]:
# 核对 Proposition 1
assert np.allclose(lam_swm, lam_opt, atol=1e-4)
assert np.allclose(p_swm,   p_opt,   atol=1e-4)
assert abs(Dmax_swm - Dmax_opt) < 1e-4
print("OK – SWM and NBP coincide")

AssertionError: 

In [7]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")

ADMM λ: [5.00117455 5.00117455 5.00117455 5.00117455 5.00117455 5.00117455
 5.00117455 5.00117455 5.00117455 4.98942907]
ADMM p: [0.59219164 0.88186133 1.06606727 1.90996514 1.95691297 1.24604279
 0.75244789 1.6644493  1.94300128 0.48245302]
ADMM Dmax: 0.9860397272183473
ADMM r: [20.24467713 20.74783153 20.3076497  20.60730078 22.62241895 21.04364342
 20.04330502 21.95985595 21.61121012 20.14225114]


In [8]:
print("social welfare:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("sum lambda:", np.sum(lamb_admm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_admm[i],lamb_admm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_admm, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_admm == max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_admm[i], lamb_admm[i]) <= Dmax_admm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_admm <= esp.D0)
print("pn<=Fn:", all(p_admm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lamb_admm[i] <= p_admm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_admm), "Q_star:", esp.Q(Dmax_admm))
print("rn>=Ln:", all(r_admm[i] >= md.Ln(p_admm[i]) for i, md in enumerate(MDs)))

social welfare: 236.5604053699815
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.4406401804726354', 'MD 2 Dn: 0.17581244763693277', 'MD 3 Dn: 0.13501352355194807', 'MD 4 Dn: 0.06381728806602266', 'MD 5 Dn: 0.06396845431144856', 'MD 6 Dn: 0.12047909112770416', 'MD 7 Dn: 0.24106938540521278', 'MD 8 Dn: 0.07324820580289378', 'MD 9 Dn: 0.06392309481931077', 'MD 10 Dn: 0.9778687470051407']
Dmax: 0.9860397272183473  D0: 5
max Dn: 0.9778687470051407
Dmax == max Dn: False
Dn<=Dmax: True
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 209.3301437290105 Q_star: 248.75434741247824
rn>=Ln: True


In [9]:
print("Nash Product Opt:",model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print("Nash Product SWM:",model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print("Nash Product ADMM:",model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))

Nash Product Opt: 37.62905234462459
Nash Product SWM: 34.947377431549235
Nash Product ADMM: 37.161857633415764


In [10]:
print("SW Opt:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("SW SWM:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("SW ADMM:",model.social_welfare(esp, MDs, lamb_admm, p_admm))

SW Opt: 245.95202135803618
SW SWM: 196.6975155148849
SW ADMM: 236.5604053699815
